In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from sklearn.metrics import accuracy_score
import numpy as np

df = pd.read_csv('D1.csv')
df.dropna(subset=['Intensity (1-3)'], inplace=True)

df

,Entries (What does the text say?),Classification,Intensity (1-3),Unnamed: 3
0,I participated in several business competition...,Business,3.0,NaN
1,"I really want to learn more about robotics, I ...",Robotics,3.0,NaN
2,I recently started learning about artifical in...,Machine Learning,1.0,NaN
3,I've been painting for about a year and want t...,Arts,2.0,NaN
4,I just heard about consulting and I would like...,Consulting,1.0,NaN
...,...,...,...,...
486,I'm just starting to learn about the blockchai...,Blockchain,1.0,NaN
487,I'm a bilangual student looking to learn more ...,Language,2.0,NaN
488,I'm a professional eSports member and I'm look...,Gaming,2.0,NaN
489,I'm looking to learn more about the medical field,Medicine,1.0,NaN


In [17]:
!pip install --upgrade tensorflow

  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [5]:

queries = df['Entries (What does the text say?)'].values
intensity = df['Intensity (1-3)'].values

X_train, X_test, y_train, y_test = train_test_split(queries, intensity, test_size=0.2, random_state=42)

from keras.utils import to_categorical

y_train = y_train - 1
y_test = y_test - 1

y_train_encoded = to_categorical(y_train, num_classes=3)
y_test_encoded = to_categorical(y_test, num_classes=3)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = max(len(x) for x in X_train_seq)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

def create_model(optimizer='adam', hidden_layers=1, units=24, input_dim=100000, max_length=max_length):
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=16, input_length=max_length),
        GlobalAveragePooling1D(),
    ])
    decrement = units
    for i in range(hidden_layers):
        model.add(Dense(decrement, activation='relu'))
        decrement -= 6
    model.add(Dense(3, activation='softmax'))  # Adjust for 5-class classification
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


keras_classifier = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('model', keras_classifier)
])

# Define the hyperparameters grid
param_grid = {
    'model__optimizer': ['adam', 'rmsprop'],
    'model__hidden_layers': [1, 2],
    'model__units': [24, 32, 64]
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train_padded, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

test_score = grid_search.score(X_test_padded, y_test)
print("Test accuracy (on test data):", test_score)


NameError: name 'KerasClassifier' is not defined